In [31]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

create_data_here = '/Users/anirudhkrishna/GitHub/FormulaData/csv-data/'

In [32]:
def parse_driver_name(row):
    words = row.split()
    parsed_string = ' '.join(words[:-1])
    return parsed_string

In [33]:
fastest_laps = pd.DataFrame()

for year in range(1950, 2024):
    races_url = f'https://www.formula1.com/en/results.html/{year}/races.html'
    response = requests.get(races_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    race_links = []
    filter_links = soup.find_all('a', attrs={'class': 'resultsarchive-filter-item-link FilterTrigger'})
    for link in filter_links:
        href = link.get('href')
        if f'/en/results.html/{year}/races/' in href:
            race_links.append(href)

    year_df = pd.DataFrame()

    round_num = 1
    for race_link in race_links:
        fastest_lap_link = race_link.replace('race-result.html', 'fastest-laps.html')
        try:
            # print(f'https://www.formula1.com{fastest_lap_link}')
            df = pd.read_html(f'https://www.formula1.com{fastest_lap_link}')[0]
            # df = df[['Pos', 'Driver', 'Lap', 'Time of day', "Time", "Avg Speed"]]
            df = df[['Pos', 'Driver', 'Lap', "Time"]]

        except:
            continue
        df['season'] = year
        df['round'] = round_num
        df = df.loc[:, ~df.columns.str.contains('Unnamed')]
        year_df = pd.concat([year_df, df], ignore_index=True)
        round_num += 1

    fastest_laps = pd.concat([fastest_laps, year_df], ignore_index=True)

fastest_laps.rename(columns={'Driver':'driver_name','Pos': 'fastest_lap_pos', 'Lap':'fastest_lap_number', 'Time of day':'time_of_day', 'Time':'lap_time', 'Avg Speed': 'avg_speed'}, inplace=True)
print(fastest_laps.shape)


   Pos             Driver  Lap      Time
0    1  Nino  Farina  FAR    2  1:50.600
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN  NaN  1:51.000
   Pos                Driver  Lap  Time
0    1  Johnnie Parsons  PAR  NaN   NaN
   Pos             Driver  Lap      Time
0    1  Nino  Farina  FAR    8  2:41.600
   Pos             Driver  Lap      Time
0    1  Nino  Farina  FAR   18  4:34.100
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN   52  2:35.600
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN    7  2:00.000
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN   33  2:51.100
   Pos             Driver  Lap      Time
0    1  Lee  Wallard  WAL   23  1:07.260
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN   10  4:22.100
   Pos                    Driver  Lap      Time
0    1  Juan Manuel  Fangio  FAN   32  2:27.800
   Pos          

In [34]:
fastest_laps["driver_name"] = fastest_laps["driver_name"].apply(parse_driver_name)

In [35]:
print(fastest_laps.head())

   fastest_lap_pos         driver_name  fastest_lap_number  lap_time  season  \
0                1         Nino Farina                 2.0  1:50.600    1950   
1                1  Juan Manuel Fangio                 NaN  1:51.000    1950   
2                1     Johnnie Parsons                 NaN       NaN    1950   
3                1         Nino Farina                 8.0  2:41.600    1950   
4                1         Nino Farina                18.0  4:34.100    1950   

   round  
0      1  
1      2  
2      3  
3      4  
4      5  


In [37]:
print(fastest_laps.tail())

       fastest_lap_pos      driver_name  fastest_lap_number  lap_time  season  \
15694               16  Valtteri Bottas                52.0  1:31.852    2023   
15695               17     Yuki Tsunoda                48.0  1:32.084    2023   
15696               18    Nyck De Vries                48.0  1:32.353    2023   
15697               19  Kevin Magnussen                29.0  1:33.356    2023   
15698               20     Esteban Ocon                 4.0  1:33.941    2023   

       round  
15694     10  
15695     10  
15696     10  
15697     10  
15698     10  


In [36]:
fastest_laps.to_csv(create_data_here + 'fastest_laps.csv', index=False)